<a href="https://colab.research.google.com/github/komazawa-deep-learning/komazawa-deep-learning.github.io/blob/master/2022notebooks/2022_0422first_neural_networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# HAD データの取得
import IPython
isColab = 'google.colab' in str(IPython.get_ipython())
if isColab:
    !pip install --upgrade xlrd
    !pip install --upgrade pandas

    # HAD サンプルデータをダウンロード
    !wget 'https://files.au-1.osf.io/v1/resources/32cyp/providers/osfstorage/5fb340145502ac018d8c86ab/?zip=' -O had.zip
    !unzip had.zip
    #from google.colab import files
    #files.upload()  # ご自身の PC からファイルをアップロードして下さい

In [10]:
import os
import sklearn.datasets
import pandas as pd
import numpy as np

#X, y = sklearn.datasets.load_iris(return_X_y=True,as_frame=True)
#_X, _y = X.to_numpy(), y.to_numpy()

had_dir = '.'  if isColab else '/Users/asakawa/study/2022HAD'
had_samples = pd.read_excel(os.path.join(had_dir, 'HAD_sample_data.xls'),sheet_name='iris')
X = had_samples[['Sepal.L','Sepal.W', 'Petal.L', 'Petal.W']].to_numpy()
# sepal:萼片, petal:花弁
# `花は内側から外に向かって、雌しべ、雄しべ、花弁、萼片、そして場合によっては苞という順番で器官が並んでいます。`
# `ですので、雄しべのすぐ外にある器官が花弁で、そのさらに外にある器官が萼片ということになります。`
# https://jspp.org/hiroba/q_and_a/detail.html?id=1219 より
y = had_samples[['Species']].to_numpy()
_y = np.zeros((y.shape[0],3))
for i, __y in enumerate(y):
     _y[i,__y[0]-1] = 1
        
y = np.copy(_y)        
#y.shape[1]


In [ ]:
import numpy as np
np.set_printoptions(precision=2)

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

X = torch.Tensor(X)
y = torch.Tensor(y)
lr = 0.01

class MLP(nn.Module):
    
    def __init__(self, n_inp, n_hid, n_out=1):
        super().__init__()
        self.n_inp = n_inp
        self.n_hid = n_hid
        self.n_out = n_out
        self.hid_layer = nn.Linear(in_features =self.n_inp, 
                                   out_features=self.n_hid,
                                   bias=True)
        self.out_layer = nn.Linear(self.n_hid, self.n_out)
        self.sigmoid = torch.nn.Sigmoid()
        
    def forward(self, x):
        x = self.hid_layer(x)
        x = self.sigmoid(x)
        x = self.out_layer(x)
        x = self.sigmoid(x)
        return x
    
mlp = MLP(n_inp=X.shape[1], n_hid=8, n_out=y.shape[1])
#loss_f = nn.MSELoss()
#loss_f = nn.CrossEntropyLoss()
loss_f = nn.BCELoss()
#optim_f = torch.optim.SGD(mlp.parameters(),lr=lr)
optim_f = torch.optim.Adam(mlp.parameters(), lr=lr)

mlp.eval()
_y = mlp(X)
_pre_loss = loss_f(_y, y)
print(f'訓練開始前の損失値:    {_pre_loss.item():.3f}')

mlp.train()
epochs = 500
for epoch in range(epochs):
    optim_f.zero_grad()
    
    _y = mlp(X)                  # モデルに処理させて出力を得る
    loss = loss_f(_y, y)         # 損失値の計算

    if epoch % (epochs>>2) == 0: # 途中結果の出力
        print(f'エポック:{epoch:4d}: 損失値:{loss.item():.3f}')

    loss.backward()              # 誤差逆伝播
    optim_f.step()               # 重み更新。すなわち学習


In [ ]:
mlp.eval()
_y = mlp(X)
for i,z in enumerate(_y):
    print(f'{i:2d} {z.detach().numpy()}')     # 結果の出力

In [ ]:
import numpy as np                                # 必要となるライブラリの輸入
np.set_printoptions(precision=3)

had_dir = '.'  if isColab else '/Users/asakawa/study/2022HAD'
had_samples = pd.read_excel(os.path.join(had_dir, 'HAD_sample_data.xls'),sheet_name='iris')
X = had_samples[['Sepal.L','Sepal.W', 'Petal.L', 'Petal.W']].to_numpy()
y = had_samples[['Species']].to_numpy()
_y = np.zeros((y.shape[0],3))
for i, y1 in enumerate(y):
     _y[i,y1[0]-1] = 1

y = np.copy(_y)        

lr, N_hid = 0.002, 8                              # lr: 学習率, N_hid: 中間層数
Wh = np.random.random((X.shape[1], N_hid)) - 1/2  # 入力層から中間層への結合係数の初期化
Wo = np.random.random((N_hid, y.shape[1])) - 1/2  # 中間層から出力層への結合係数の初期化
epochs = 500
for t in range(epochs):                           # 繰り返し
    H  = np.tanh(np.dot(X, Wh))                   # 入力層から中間層への計算。ハイパータンジェント関数
    _y = 1/(1. + np.exp(-(np.dot(H, Wo))))        # 中間層から出力層への計算。シグモイド関数
    Dy = (y - _y) * (_y * (1. - _y))              # 誤差の微分
    DH = Dy.dot(Wo.T) * (1. - H ** 2)             # 誤差逆伝播
    Wo += lr * H.T @ Dy
    Wh += lr * X.T @ DH                           # 中間層から入力層への重み更新
    if t % (epochs>>2) == 0:
        print(np.array((y-_y)**2).mean())

for i,z in enumerate(_y):
    print(f'{i:2d} {z}')                          # 結果の出力

#### 積分発火モデル

ニューロンが振動数符号化法のみを利用している --- すなわちニューラルネットワークにおけるすべての情報はニューロンの発火頻度によって伝達される --- ことを仮定する。
すなわち，このニューロンに到達する信号を単位時間あたりで等しく貢献するものと考える。
このような記述の仕方は integrate--and--fire (積分発火) モデルと呼ばれる。

一つのニューロンの振る舞いは，n 個のニューロンから入力を受け取って出力を計算する多入力，1 出力の情報処理素子である (下図 1)。

$n$ 個の入力信号を $x_1, x_2, \cdots, x_n$ とし、$i$ 番目の軸索に信号が与えられたとき、この信号 1 単位によって変化する膜電位の量をシナプス荷重(または結合荷重, 結合強度とも呼ばれる) といい $w_i$ と表記する。
抑制性のシナプス結合については $w_i < 0$, 興奮性の結合については $w_i > 0$ である。
このニューロンのしきい値を $h$, 膜電位の変化を $u$, 出力信号を $z$ とする。

<center>
<img src="https://komazawa-deep-learning.github.io/assets/formal_neuron.svg"><br/>
<!-- <img src="figures/formal_neuron.svg"><br/> -->
図 1. ニューロンの模式図
</center>

出力信号 $z$ は次式で表される:
$$
z=f(\mu)=f\left(\sum_{i=1}^{n}w_{i}x_{i}-h\right).\tag{1}
$$
$f(\mu)$ は出力関数であり、$0\le f(\mu)\le1$ の連続量を許す場合や、0 または 1 の値しかとらない場合などがある。
連想記憶などを扱うときなどは $-1\le f(\mu)\le 1$ とする場合もある。

- 上図で，$f$ がなければ，$z=\sum wx - h$ となり，回帰と同じである。
- すなわちニューラルネットワークのもっとも簡単な形は，統計学の用語では，**回帰 regression** である。
- 統計学においては，回帰，この場合，線形回帰になる，にかかわる変数 $x_{i}$ 
- 回帰の中を複雑にしていくことで，データに適合させようとする努力が，データサイエンスであり，機械学習であり，ニューラルネットワークであると言える。


## マッカロック・ピッツの形式ニューロン

信号入力の荷重和 
$$
\mu = \sum_{i=1}^{n}w_{i}x_{i},\tag{2}
$$
に対して、出力 $z$ は $u$ がしきい値 $h$ を越えたときに $1$, そうでなければ 0 を出力するモデルのことをマッカロックとピッツの形式ニューロン (formal neuron) と呼ぶ。
マッカロック・ピッツの形式ニューロンは神経細胞の振る舞いを記述するもっとも古く、単純な神経細胞のモデルであるが、現在でも用いられることがある。

$$
z = \left\{ \begin{array}{ll}
 1, & \mbox{$u > 0$ のとき} \\
 0, & \mbox{それ以外}
 \end{array}
\right.
$$

とすればマッカロック・ピッツのモデルは次式で表すことができる

$$
z=\mathbb{1}\left(\sum_{i=1}^nw_ix_i -h\right)\tag{3}
$$

式中の $\mathbb{1}$ は数字ではなく 上式で表される関数の意味である。
このモデルは、単一ニューロンのモデルとしてではなく、ひとまとまりのニューロン群の動作を示すモデルとしても用いることがある。

---

### 出力が連続関数の場合

時間 $t$ における入力信号は $x_i(t)$ は $i$ 番目のシナプスの興奮伝達の時間 $t$ 付近での平均ととらえることができる。
すると、最高頻度の出力を 1, 最低(興奮無し)を 0 と規格化できると考えて $0\le f(\mu) \le1$ とする。
入出力関係は $f$ を用いて

$$
z = f(\mu) = f\left(\sum w_ix_i(t)-h\right)\tag{4}
$$

のように表現される。
このモデルは、ニューロン集団の平均活動率ととらえることもできる。

良く用いられる出力関数 $f$ の形としては、$\mu = \sum w_ix_i -h$ として、

$$
f(\mu) = \frac{1}{1+e^{-\mu}},\tag{5}
$$

や

$$
f(\mu) = \tanh(\mu)\tag{6}, 
$$

ただし $\mu = \sum w_{i}x_{i} -h$ などが使われることが多い。
(5) 式および (6) 式は、入力信号の重みつき荷重和 $\mu$ としてニューロンの活動が定まることを示している。
後述するバックプロパゲーション則で必要となるので、$\mu$ の微小な変化がニューロンの活動どのような影響を与えるか調べるために (5) 式 および (6) 式 を $\mu$ で微分することを考える。

$$
f(x) = \frac{1}{1+e^{-x}},
$$

を $x$ について微分すると

$$
\frac{df}{dx} = f(x)\left(1- f(x)\right)
$$

$$
f(x) = \tanh{x} = \frac{e^{x}-e^{-x}}{e^{x}+e^{-x}}
$$

を $x$ について微分すると

$$
\frac{df}{dx} = 
1 - \tanh^2x = 1 - \left(f(x)\right)^2
$$

$\tanh$ は双曲線関数である。

以降では表記を簡単にするために線形数学の表記、すなわちベクトルと行列による
表記方法を導入する。$n$ 個の入力信号の組 $\left(x_1, x_2, \cdots, x_n\right)$ をまと
めて $\mathbf{x}$ のようにボールド体で表す。本章では一貫してベクトル表記には小
文字のボールド体を、行列には大文字のボールド体を用いることにする。例えば 
$n$ 個の入力信号の組 $(x_1, x_2, \cdots, x_n)=\mathbf{x}$ に対して、同数の結合
荷重 $\left(w_1, w_2, \cdots, w_n\right)=\mathbf{w}$ が存在するので、加算記号 $\sum$ を使っ
て表現していた任意のニューロンへの全入力 $\mu=\sum w_{i}x_{i}$ はベクトルの内
積を用いて $\mu=(\mathbf{w}\cdot\mathbf{x})$ と表現される。

なお、横一行のベクトルを行ベクトル、縦一列のベクトルを列ベクトルと呼ぶことがある。
ここでは行ベクトルと混乱しないように、必要に応じて列ベクトルを表現する際には $\{x_1, x_2, \cdots, x_n\}^{\top}=\mathbf{x}$ とベクトルの肩に $\top$ を使って表現することもある。

そして、これらをベクトル表現や行列表現で表せば、表記も簡単になり、行列の演算法則を活用することもできる。
そのため、ニューラルネットワークに関する文献でも行列表現が用いられることが多い。

図のような単純な 2  層の回路を例に説明する。

<center>
<img src="https://komazawa-deep-learning.github.io/assets/matrix-notation.svg"><br/>
<!-- <img src="figures/matrix-notation.svg"><br/> -->
ネットワークの行列表現
</center>

わかりやすいように図と対応させながら、対応する行列表現とシグマ記号による表記を併記するので、よく理解した上で先に読み進んでいただきたい。
なお、どうしても行列表現にはなじめないという読者は、行列表現の箇所だけをとばして読んでもある程度はわかるよう記述するつもりである。

それでは、まず図 4 のような単純な 2 層のネットワークを例に説明しよう。
図には、３つの入力素子 (ユニットと呼ばれることもある) 
と２つの出力素子の活性値（ニューロンの膜電位に相当する）

$x_{1}, x_{2}, x_{3}$ と $y_{1}, y_{2}$，および入力素子と出力素子の結合強度を表す $w_{11}, w_{12}, \cdots, w_{32}$ が示されている。
これらの記号をベクトル $\mathbf{x}$, $\mathbf{y}$ と行列 $\mathbf{w}$ を使って表すと $\mathbf{y}=\mathbf{Wx}$ となる。

図\ref{fig:matrix-notation.eps}の場合、ベクトルと行列の各要素を書き下せば、
$$
\left(\begin{array}{l}y_{1}\\
y_2\\
\end{array}\right)
=\left(
\begin{array}{lll}
w_{11}&w_{12}&w_{13}\\
w_{21}&w_{22}&w_{23}
\end{array}
\right)
\left(
\begin{array}{l}
x_1\\
x_2\\
x_3\\
\end{array}
\right)
$$
のようになる。　　

---

行列の積は、左側の行列の $i$ 行目の各要素と右側の行列（ベクトルは１列の行列でもある）の $i$ 列目の各要素とを掛け合わせて合計することなので、以下のような、加算記号を用いた表記と同じである。

$$
\begin{array}{lllll}
y_1 &=& w_{11}x_1 + w_{12}x_2 + w_{13}x_3 &=&\sum_i w_{1i} x_i\\
y_2 &=& w_{21}x_1 + w_{22}x_2 + w_{23}x_3 &=&\sum_i w_{2i} x_i\\
\end{array}
$$

これを、$m$ 個の入力ユニットと $n$ 個の出力ユニットの場合に一般化すれば、
以下のようになる。


単純な 2 層のネットワークを考える。
$i$ 番目の出力層の各ニューロンの膜電位 $y_i,(i=1,2,\cdots,n)$ をまとめて $\mathbf{y}$ とベクトル表現し、同様に入力層も $\mathbf{x}$ とする。
ニューロン $x_{j}$ から ニューロン $y_{i}$ へのシナプス結合強度を $w_{ij}$ と表現すれば、入力層から出力層への関係は

$$
\left(\begin{array}{l}
y_1 \\ y_2 \\ \vdots \\ y_n \end{array}\right)  = 
\left(\begin{array}{llll}
w_{11} & w_{12} & \cdots & w_{1m} \\
w_{21} & w_{22} & \cdots & w_{2m} \\
\vdots &        & \ddots & \vdots \\
w_{n1} & w_{n2} & \cdots & w_{nm} 
\end{array}\right)
\left(\begin{array}{l}
x_1 \\ x_2 \\ \vdots \\ x_m \end{array}\right) \\
\mathbf{y} = \mathbf{Wx}
$$

と表現できる。
しきい値の扱いについては、常に 1 を出力する仮想的なニューロン$x_0=1$を
考えて $W$ に組み込むことも可能である。


実際の出力は $\mathbf{y}$ の各要素に対して 

$$
f(y) = \frac{1}{1+e^{-y}},
$$

のような非線型変換を施すことがある。

階層型のネットワークにとっては上式，非線型変換が本質的な役割を果たす。
なぜならば、こうした非線形変換がなされない場合には、ネットワークの構造が何層になったとしても、この単純なシナプス結合強度を表す行列を $\mathbf{W}_{i}$
($i=1,\cdots, p$) としたとき、$\mathbf{W} = \prod_{i=1}^p \mathbf{W}_{i}$ と置くことによって本質的には 1 層のネットワークと等価になるからである。

$$
\mathbf{y} = \mathbf{W}_{p}\mathbf{W}_{p-1}\cdots\mathbf{W}_{1}\mathbf{y}=\left(\prod_{i=1}^p\mathbf{W}_{i}\right)\mathbf{y}.
$$


<img src="https://komazawa-deep-learning.github.io/assets/xor.svg"><br/>

<img src="https://komazawa-deep-learning.github.io/assets/xor-graph.svg"><br/>

